In [ ]:
### ANTES DE EXECUTAR GARANTIR QUE O BANCO NEO4J ESTÁ NO AR ###
### ALTERAR A VARIAVEL DATA #######

import seaborn as sns
import pandas as pd
from neo4j import GraphDatabase
import hashlib
from dotenv import load_dotenv
import os
from datetime import datetime


load_dotenv()
# Data da Consulta
date = "11/02/2025"

# Converte para um objeto datetime
date_obj = datetime.strptime(date, "%d/%m/%Y")

# Converte para o formato yyyy-mm-dd
formatted_date = date_obj.strftime("%Y-%m-%d")

# Comando Python que você deseja executar
cmd_special = "python showusage.py -ds " + formatted_date  + " -report SPECIAL -t ALMEIDA -csv"

cmd_compartment = "python showusage.py -ds " + formatted_date  + "  -report COMPARTMENT -t ALMEIDA -csv"


os.system(cmd_special)

os.system(cmd_compartment)


def computeMD5hash(my_string):
    m = hashlib.md5()
    m.update(my_string.encode('utf-8'))
    return m.hexdigest()

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI =os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD'))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session(database="neo4j")

file_compartment = "usage_compartments.csv"
file_special = "usage_special.csv"
file_name = "dados.csv"
df_compartment = pd.read_csv(file_compartment)

df_special = pd.read_csv(file_special)

df_compartment = df_compartment[["Compartment Path", "Service", "Currency", "Cost"]] # as duas primeiras colunas do dataframe

df_special = df_special.iloc[:, 0:4] # as quatro primeiras colunas do dataframe

df_compartment.set_index('Service')
df_special.set_index('Service')

df = df_special.merge(df_compartment, on='Service', how='left')


query = '''
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

'''

df.to_csv(file_name, sep=';', encoding='utf-8', index=False, header=True)

for key, value in df.iterrows():
    data = {
        'Service': value['Service'],
        'Region': value['Region'],
        'Product_SKU': value['Product SKU'],
        'Product_Name': value['Product Name'],
        'Compartment_Path': value['Compartment Path'],
        'Currency': value['Currency'],
        'Cost': value['Cost'],
        'Cost_id': computeMD5hash(value['Product SKU'] + value['Compartment Path']),
        'Date': date
    }
    records, summary, keys = driver.execute_query(query, data=data, )
    print("The query `{query}` returned {records_count} records in {time} ms.".format(query=summary.query, records_count=len(records),time=summary.result_available_after, ))



# session/driver usage

session.close()
driver.close()

In [ ]:
## CARREGANDO ARQUIVO COST USAGE

from neo4j import GraphDatabase
import csv
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

# Define the CSV file path
csv_file_path = '../data/oracle/reports_cost-csv_0001000001812618-00001.csv'

# Define the Neo4j connection details
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI =os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD'))

# Define the Neo4j driver
driver = GraphDatabase.driver(URI, auth=AUTH)


query = '''
    unwind $data as row
    MERGE (lineItem:LineItem {
        referenceNo: row.referenceNo,
        tenantId: row.tenantId,
        intervalUsageStart: row.intervalUsageStart,
        intervalUsageEnd: row.intervalUsageEnd,
        isCorrection: row.isCorrection,
        backreferenceNo: row.backreferenceNo
    })
    MERGE (product:Product {
        service: row.service,
        compartmentId: row.compartmentId,
        compartmentName: row.compartmentName,
        region: row.region,
        availabilityDomain: row.availabilityDomain,
        resourceId: row.resourceId,
        description: row.description
    })
    MERGE (usage:Usage {
        billedQuantity: row.billedQuantity,
        billedQuantityOverage: row.billedQuantityOverage,
        attributedUsage: row.attributedUsage
    })
    MERGE (cost:Cost {
        subscriptionId: row.subscriptionId,
        productSku: row.productSku,
        unitPrice: row.unitPrice,
        unitPriceOverage: row.unitPriceOverage,
        myCost: row.myCost,
        myCostOverage: row.myCostOverage,
        currencyCode: row.currencyCode,
        billingUnitReadable: row.billingUnitReadable,
        skuUnitDescription: row.skuUnitDescription,
        overageFlag: row.overageFlag,
        attributedCost: row.attributedCost
    })
    MERGE (tags:Tags {
        CreatedBy: row.CreatedBy,
        InstanceId: row.InstanceId,
        OKEclusterName: row.OKEclusterName,
        OKEnodePoolName: row.OKEnodePoolName,
        OracleTagsCreatedBy: row.OracleTagsCreatedBy,
        OracleTagsCreatedOn: row.OracleTagsCreatedOn,
        Template: row.Template,
        loadbalancer: row.loadbalancer,
        orclContainerengineCluster: row.orclContainerengineCluster,
        orclContainerengineNodePool: row.orclContainerengineNodePool,
        orclContainerengineNodeType: row.orclContainerengineNodeType
    })
    MERGE (lineItem)-[:HAS_PRODUCT]->(product)
    MERGE (lineItem)-[:HAS_USAGE]->(usage)
    MERGE (lineItem)-[:HAS_COST]->(cost)
    MERGE (lineItem)-[:HAS_TAGS]->(tags)
    '''

def main():
    df = pd.read_csv(csv_file_path)
    df = df.rename(columns={'lineItem/referenceNo': 'referenceNo',
                       'lineItem/tenantId': 'tenantId',      
                       'lineItem/intervalUsageStart': 'intervalUsageStart',
                       'lineItem/intervalUsageEnd': 'intervalUsageEnd',
                       'product/service': 'service',
                       'product/compartmentId': 'compartmentId',
                       'product/compartmentName': 'compartmentName',
                       'product/region': 'region',
                       'product/availabilityDomain': 'availabilityDomain',
                       'product/resourceId': 'resourceId',
                       'usage/billedQuantity': 'billedQuantity',
                       'usage/billedQuantityOverage': 'billedQuantityOverage',
                       'cost/subscriptionId': 'subscriptionId',
                       'cost/productSku': 'productSku',
                       'product/Description': 'description',
                       'cost/unitPrice': 'unitPrice',
                       'cost/unitPriceOverage': 'unitPriceOverage',
                       'cost/myCost': 'myCost',
                       'cost/myCostOverage': 'myCostOverage',
                       'cost/currencyCode': 'currencyCode',
                       'cost/billingUnitReadable': 'billingUnitReadable',
                       'cost/skuUnitDescription': 'skuUnitDescription',
                       'cost/overageFlag': 'overageFlag',
                       'lineItem/isCorrection':'isCorrection',
                       'lineItem/backreferenceNo':'backreferenceNo',
                       'cost/attributedCost':'attributedCost',
                       'usage/attributedUsage':'attributedUsage',
                       'tags/CreatedBy':'CreatedBy',
                       'tags/InstanceId':'InstanceId',
                       'tags/OKEclusterName':'OKEclusterName',
                       'tags/OKEnodePoolName':'OKEnodePoolName',
                       'tags/Oracle-Tags.CreatedBy':'OracleTagsCreatedBy',
                       'tags/Oracle-Tags.CreatedOn':'OracleTagsCreatedOn',
                       'tags/Template':'Template',
                       'tags/loadbalancer':'loadbalancer',
                       'tags/orcl-containerengine.Cluster':'orclContainerengineCluster',
                       'tags/orcl-containerengine.NodePool':'orclContainerengineNodePool',
                       'tags/orcl-containerengine.NodeType':'orclContainerengineNodeType'
                       })
    df.fillna('Empty', inplace=True)
    df.replace('', 'Empty', inplace=True)
    for key, value in df.iterrows():
        data = {
            'referenceNo': value['referenceNo'],
            'tenantId': value['tenantId'],
            'intervalUsageStart': value['intervalUsageStart'],
            'intervalUsageEnd': value['intervalUsageEnd'],
            'service': value['service'],
            'compartmentId': value['compartmentId'],
            'compartmentName': value['compartmentName'],
            'region': value['region'],
            'availabilityDomain': value['availabilityDomain'],
            'resourceId': value['resourceId'],
            'billedQuantity': value['billedQuantity'],
            'billedQuantityOverage': value['billedQuantityOverage'],
            'subscriptionId': value['subscriptionId'],
            'productSku': value['productSku'],
            'description': value['description'],
            'unitPrice': value['unitPrice'],
            'unitPriceOverage': value['unitPriceOverage'],
            'myCost': value['myCost'],
            'myCostOverage': value['myCostOverage'],
            'currencyCode': value['currencyCode'],
            'billingUnitReadable': value['billingUnitReadable'],
            'skuUnitDescription': value['skuUnitDescription'],
            'overageFlag': value['overageFlag'],
            'isCorrection': value['isCorrection'],
            'backreferenceNo': value['backreferenceNo'],
            'attributedCost': value['attributedCost'],
            'attributedUsage': value['attributedUsage'],
            'CreatedBy': value['CreatedBy'],
            'InstanceId': value['InstanceId'],
            'OKEclusterName': value['OKEclusterName'],
            'OKEnodePoolName': value['OKEnodePoolName'],
            'OracleTagsCreatedBy': value['OracleTagsCreatedBy'],
            'OracleTagsCreatedOn': value['OracleTagsCreatedOn'],
            'Template': value['Template'],
            'loadbalancer': value['loadbalancer'],
            'orclContainerengineCluster': value['orclContainerengineCluster'],
            'orclContainerengineNodePool': value['orclContainerengineNodePool'],
            'orclContainerengineNodeType': value['orclContainerengineNodeType']
        }
        print(data)
        records, summary, keys = driver.execute_query(query, data=data, )
        print("The query `{query}` returned {records_count} records in {time} ms.".format(query=summary.query, records_count=len(records),time=summary.result_available_after, ))

if __name__ == "__main__":
    main()

In [7]:
## DESCOMPACTANDO MES DE ARQUIVOS 

import os
import glob
import gzip
import shutil


# Specify the directory to search

folder = "202411"
directory = 'c:/Users/rodrigo.almeida/Documents/GitHub/BudgetReports/data/oracle/11'
directory_saida = "c:/Users/rodrigo.almeida/Documents/GitHub/BudgetReports/data/oracle/" + folder


def unzip_gz_file(gz_file_path, output_file_path):
    with gzip.open(gz_file_path, 'rb') as f_in:
        with open(output_file_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

def find_gz_files(directory):
    # Use glob to find all .gz files recursively
    gz_files = glob.glob(os.path.join(directory, '**', '*.gz'), recursive=True)
    return gz_files

# Get the list of .gz files
gz_files = find_gz_files(directory)

# Print the list of .gz files
count = 1
for file in gz_files:
    output_file_path = directory_saida + "/" + folder + "-" + str(count) + ".csv"
    unzip_gz_file(file, output_file_path)
    count = count + 1

In [29]:
## CARREGANDO BANCO NEO4J
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import pandas as pd
import json
import glob
import time
import nest_asyncio
import warnings

warnings.filterwarnings('ignore')
nest_asyncio.apply()
load_dotenv()

# Define the CSV file path
periodo = "202501"


dir_arquivos = "c:/Users/rodrigo.almeida/Documents/GitHub/BudgetReports/data/oracle/" + periodo


# Define the Neo4j connection details
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI =os.getenv('NEO4J_URI')
AUTH = (os.getenv('NEO4J_USER'), os.getenv('NEO4J_PASSWORD'))

# Define the Neo4j driver
driver = GraphDatabase.driver(URI, auth=AUTH)

# {""Oracle-Tags.CreatedBy"":""domain_dest/gabriel.doliveira@dataprev.gov.br"",""Oracle-Tags.CreatedOn"":""2024-12-20T14:30:25.443Z"",""Template"":"""",""loadbalancer"":"""",""orcl-containerengine.Cluster"":"""",""orcl-containerengine.NodePool"":"""",""orcl-containerengine.NodeType"":""""}

query = '''
    unwind $data as row
    MERGE (compartment:Compartment {        
        oci_CompartmentId: row.oci_CompartmentId,
        oci_CompartmentName: row.oci_CompartmentName,                                    
        Region: row.Region,
        SubAccountId: row.SubAccountId,
        SubAccountName: row.SubAccountName

    })
    MERGE (product:Product {
        ResourceName: row.ResourceName,
        ResourceType: row.ResourceType,
        ServiceCategory: row.ServiceCategory,
        ServiceName: row.ServiceName,
        SkuId: row.SkuId
    })
    MERGE (usage:Usage {
        BilledCost: row.BilledCost,
        oci_AttributedUsage: row.oci_AttributedUsage,
        BillingAccountName: row.BillingAccountName,
        BillingCurrency: row.BillingCurrency,
        BillingPeriodEnd: row.BillingPeriodEnd,
        BillingPeriodStart: row.BillingPeriodStart,
        oci_BilledQuantityOverage: row.oci_BilledQuantityOverage,
        ChargeCategory: row.ChargeCategory,
        ChargeDescription: row.ChargeDescription,
        ChargeFrequency: row.ChargeFrequency,
        ChargeSubcategory: row.ChargeSubcategory,
        UsageQuantity: row.UsageQuantity,
        UsageUnit: row.UsageUnit
    })
    MERGE (cost:Cost {
        EffectiveCost: row.EffectiveCost,
        InvoiceIssuer: row.InvoiceIssuer,
        ListCost: row.ListCost,
        ListUnitPrice: row.ListUnitPrice,
        oci_UnitPriceOverage: row.oci_UnitPriceOverage,
        oci_CostOverage: row.oci_CostOverage,
        oci_AttributedCost: row.oci_AttributedCost,
        PricingCategory: row.PricingCategory,
        PricingQuantity: row.PricingQuantity,
        PricingUnit: row.PricingUnit,
        Provider: row.Provider,
        Publisher: row.Publisher
    })
    MERGE (tags:Tags {
        oracleTagsCreatedBy: row.oracleTagsCreatedBy,
        oracleTagsCreatedOn: row.oracleTagsCreatedOn,
        template: row.template,
        loadbalancer: row.loadbalancer,
        orclcontainerengineCluster: row.orclcontainerengineCluster,
        orclcontainerengineNodePool: row.orclcontainerengineNodePool,
        orclContainerengineNodeType: row.orclcontainerengineNodeType
    })    
    MERGE (compartment)-[:HAS_PRODUCT {periodo: row.Periodo}]->(product)
    MERGE (product)-[:HAS_USAGE {periodo: row.Periodo, oci_CompartmentName: row.oci_CompartmentName}]->(usage)
    MERGE (product)-[:HAS_COST {periodo: row.Periodo, oci_CompartmentName: row.oci_CompartmentName, EffectiveCost: row.EffectiveCost }]->(cost)
    MERGE (product)-[:HAS_TAGS {periodo: row.Periodo, oci_CompartmentName: row.oci_CompartmentName}]->(tags)
    '''

async def parse_tags(tags_str):
    try:
        return json.loads(tags_str)
    except json.JSONDecodeError:
        return {}

async def cargaGrafo(csv_file_path):
    df = pd.read_csv(csv_file_path)
    # Ensure the Tags column is treated as strings
    df['Tags'] = df['Tags'].astype(str)
    df.fillna('Empty', inplace=True)
    df.replace('', 'Empty', inplace=True)
    data = []
    count = 1
    for _, row in df.iterrows():
        tags = await parse_tags(row['Tags'])
        row_data = {
            'Periodo': periodo, 
            'AvailabilityZone': row['AvailabilityZone'],
            'BilledCost': row['BilledCost'],
            'BillingAccountId': row['BillingAccountId'],
            'BillingAccountName': row['BillingAccountName'],
            'BillingCurrency': row['BillingCurrency'],
            'BillingPeriodEnd': row['BillingPeriodEnd'],
            'BillingPeriodStart': row['BillingPeriodStart'],
            'ChargeCategory': row['ChargeCategory'],
            'ChargeDescription': row['ChargeDescription'],
            'ChargeFrequency': row['ChargeFrequency'],
            'ChargeSubcategory': row['ChargeSubcategory'],
            'CommitmentDiscountCategory': row['CommitmentDiscountCategory'],
            'CommitmentDiscountId': row['CommitmentDiscountId'],
            'CommitmentDiscountName': row['CommitmentDiscountName'],
            'CommitmentDiscountType': row['CommitmentDiscountType'],
            'EffectiveCost': row['EffectiveCost'],
            'InvoiceIssuer': row['InvoiceIssuer'],
            'ListCost': row['ListCost'],
            'ListUnitPrice': row['ListUnitPrice'],
            'PricingCategory': row['PricingCategory'],
            'PricingQuantity': row['PricingQuantity'],
            'PricingUnit': row['PricingUnit'],
            'Provider': row['Provider'],
            'Publisher': row['Publisher'],
            'Region': row['Region'],
            'ResourceId': row['ResourceId'],
            'ResourceName': row['ResourceName'],
            'ResourceType': row['ResourceType'],
            'ServiceCategory': row['ServiceCategory'],
            'ServiceName': row['ServiceName'],
            'SkuId': row['SkuId'],
            'SkuPriceId': row['SkuPriceId'],
            'SubAccountId': row['SubAccountId'],
            'SubAccountName': row['SubAccountName'],
            'UsageQuantity': row['UsageQuantity'],
            'UsageUnit': row['UsageUnit'],
            'oci_ReferenceNumber': row['oci_ReferenceNumber'],
            'oci_CompartmentId': row['oci_CompartmentId'],
            'oci_CompartmentName': row['oci_CompartmentName'],
            'oci_OverageFlag': row['oci_OverageFlag'],
            'oci_UnitPriceOverage': row['oci_UnitPriceOverage'],
            'oci_BilledQuantityOverage': row['oci_BilledQuantityOverage'],
            'oci_CostOverage': row['oci_CostOverage'],
            'oci_AttributedUsage': row['oci_AttributedUsage'],
            'oci_AttributedCost': row['oci_AttributedCost'],
            'oci_BackReferenceNumber': row['oci_BackReferenceNumber'],
            'oracleTagsCreatedBy': tags.get('Oracle-Tags.CreatedBy', 'Empty'),
            'oracleTagsCreatedOn': tags.get('Oracle-Tags.CreatedOn', 'Empty'),
            'template': tags.get('Template', 'Empty'),
            'loadbalancer': tags.get('loadbalancer', 'Empty'),
            'orclcontainerengineCluster': tags.get('orcl-containerengine.Cluster', 'Empty'),
            'orclcontainerengineNodePool': tags.get('orcl-containerengine.NodePool', 'Empty'),
            'orclcontainerengineNodeType': tags.get('orcl-containerengine.NodeType', 'Empty')
        }
        data.append(row_data)

    with driver.session() as session:
        session.run(query, data=data)



async def main():
    # Get all CSV files in the folder
    csv_files = glob.glob(os.path.join(dir_arquivos, '*.csv'))


    # Measure the start time
    start_time = time.time()

    # Process each CSV file
    for csv_file_path in csv_files:
        await cargaGrafo(csv_file_path)

    # Measure the end time
    end_time = time.time()

    # Calculate and print the elapsed time
    elapsed_time = end_time - start_time
    print(f"Tempo total para processar todos os arquivos: {elapsed_time:.2f} segundos")



# Run the main function
await main()



Tempo total para processar todos os arquivos: 221.12 segundos
